Задание 1. Вариант №1 (с помощью функции со специальными правами заимствования)

In [9]:
import requests

def get_max_currency():
    # Получаем данные о курсах валют
    response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
    data = response.json()

    # Ищем валюту с максимальным курсом
    max_rate = 0
    max_currency_name = ''

    for currency_code, currency_info in data['Valute'].items():
            
        current_rate = currency_info['Value']/ currency_info['Nominal']
        if current_rate > max_rate:
            max_rate = current_rate
            max_currency_name = currency_info['Name']

    return max_currency_name


max_currency = get_max_currency()
print(f"Валюта с максимальным курсом: {max_currency}")

Валюта с максимальным курсом: СДР (специальные права заимствования)


Задание 1. Вариант № 2. Поскольку СДР не являются валютой, а только единицей для международных рассчетов, решим задачу с помощью класса.

In [11]:
import requests

class CurrencyAnalyzer:
    def __init__(self):
        self.api_url = 'https://www.cbr-xml-daily.ru/daily_json.js'
        self._update_data()

    def _update_data(self):
        response = requests.get(self.api_url)
        self.data = response.json()['Valute']

    def get_max_currency(self, excluded_currency=None):
        self._update_data()  # Обновляем данные перед каждым запросом

        max_rate = 0
        max_currency_name = ''

        for currency_code, currency_info in self.data.items():
            if excluded_currency and currency_info['Name'] == excluded_currency:
                continue

            current_rate = currency_info['Value']/ currency_info['Nominal'] # делим на значения ключа 'Nominal'
            if current_rate > max_rate:
                max_rate = current_rate
                max_currency_name = currency_info['Name']

        return max_currency_name

# Пример использования
analyzer = CurrencyAnalyzer()
max_currency = analyzer.get_max_currency(excluded_currency="СДР (специальные права заимствования)")
print(f"Валюта с максимальным курсом (исключая СДР): {max_currency}")

Валюта с максимальным курсом (исключая СДР): Фунт стерлингов Соединенного королевства


Задание 2 

In [71]:
import requests

class Rate:
    def __init__(self, format_='value', diff=False):
        self.format = format_
        self.diff = diff
        self.previous_rates = self._get_previous_rates()

    def _get_previous_rates(self):
        response = self._make_api_request()
        return {currency: response[currency]['Previous']/ response[currency]['Nominal'] for currency in response}

    def _make_api_request(self):
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']

    def _get_diff(self, currency):
        current_rate = self._make_api_request()[currency]['Value']/ self._make_api_request()[currency]['Nominal']
        previous_rate = self.previous_rates[currency]
        return round(current_rate - previous_rate, 6)

    def _get_format_value(self, currency):
        response = self._make_api_request()

        if currency in response:
            if self.format == 'full':
                return response[currency]

            if self.format == 'value':
                if self.diff:
                    return self._get_diff(currency)
                else:
                    return response[currency]['Value']/ response[currency]['Nominal']

        return 'Error'

    def eur(self):
        return self._get_format_value('EUR')

    def usd(self):
        return self._get_format_value('USD')
    def brl(self):
        return self._get_format_value('BRL')
    def try_(self):
        return self._get_format_value('TRY')





In [75]:
r = Rate(format_ = 'full')
r.usd()  

{'ID': 'R01235',
 'NumCode': '840',
 'CharCode': 'USD',
 'Nominal': 1,
 'Name': 'Доллар США',
 'Value': 93.0351,
 'Previous': 93.173}

In [76]:
r = Rate(diff=True)
r.usd()

-0.1379

In [77]:
r = Rate(diff=False)
r.usd()

93.0351